In [105]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


Sources: 
- https://github.com/CSAILVision/places365
- https://colab.research.google.com/drive/1iGGrigHMoCFcaneMb6g4Zqh5iZEMMuye#scrollTo=xRrZdXAqfy0X 

In [106]:
import torch
from torch.autograd import Variable as V
import torchvision.models as models
from torchvision import transforms as trn
from torch.nn import functional as F
import os
import numpy as np
import cv2
from PIL import Image
from google.colab.patches import cv2_imshow
import pandas as pd
import csv

In [107]:
 # hacky way to deal with the Pytorch 1.0 update
def recursion_change_bn(module):
    if isinstance(module, torch.nn.BatchNorm2d):
        module.track_running_stats = 1
    else:
        for i, (name, module1) in enumerate(module._modules.items()):
            module1 = recursion_change_bn(module1)
    return module


In [108]:

def load_labels():
    # prepare all the labels
    # scene category relevant
    file_name_category = 'categories_places365.txt'
    if not os.access(file_name_category, os.W_OK):
        synset_url = 'https://raw.githubusercontent.com/csailvision/places365/master/categories_places365.txt'
        os.system('wget ' + synset_url)
    classes = list()
    with open(file_name_category) as class_file:
        for line in class_file:
            classes.append(line.strip().split(' ')[0][3:])
    classes = tuple(classes)

    # indoor and outdoor relevant
    file_name_IO = 'IO_places365.txt'
    if not os.access(file_name_IO, os.W_OK):
        synset_url = 'https://raw.githubusercontent.com/csailvision/places365/master/IO_places365.txt'
        os.system('wget ' + synset_url)
    with open(file_name_IO) as f:
        lines = f.readlines()
        labels_IO = []
        for line in lines:
            items = line.rstrip().split()
            labels_IO.append(int(items[-1]) -1) # 0 is indoor, 1 is outdoor
    labels_IO = np.array(labels_IO)

    # scene attribute relevant
    file_name_attribute = 'labels_sunattribute.txt'
    if not os.access(file_name_attribute, os.W_OK):
        synset_url = 'https://raw.githubusercontent.com/csailvision/places365/master/labels_sunattribute.txt'
        os.system('wget ' + synset_url)
    with open(file_name_attribute) as f:
        lines = f.readlines()
        labels_attribute = [item.rstrip() for item in lines]
    file_name_W = 'W_sceneattribute_wideresnet18.npy'
    if not os.access(file_name_W, os.W_OK):
        synset_url = 'http://places2.csail.mit.edu/models_places365/W_sceneattribute_wideresnet18.npy'
        os.system('wget ' + synset_url)
    W_attribute = np.load(file_name_W)

    return classes, labels_IO, labels_attribute, W_attribute

In [109]:
def hook_feature(module, input, output):
    features_blobs.append(np.squeeze(output.data.cpu().numpy()))

In [110]:
def returnCAM(feature_conv, weight_softmax, class_idx):
    # generate the class activation maps upsample to 256x256
    size_upsample = (256, 256)
    nc, h, w = feature_conv.shape
    output_cam = []
    for idx in class_idx:
        cam = weight_softmax[class_idx].dot(feature_conv.reshape((nc, h*w)))
        cam = cam.reshape(h, w)
        cam = cam - np.min(cam)
        cam_img = cam / np.max(cam)
        cam_img = np.uint8(255 * cam_img)
        output_cam.append(cv2.resize(cam_img, size_upsample))
    return output_cam

In [111]:
def returnTF():
# load the image transformer
    tf = trn.Compose([
        trn.Resize((224,224)),
        trn.ToTensor(),
        trn.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    return tf

In [112]:
def load_model():
    # this model has a last conv feature map as 14x14

    model_file = 'wideresnet18_places365.pth.tar'
    if not os.access(model_file, os.W_OK):
        os.system('wget http://places2.csail.mit.edu/models_places365/' + model_file)
        os.system('wget https://raw.githubusercontent.com/csailvision/places365/master/wideresnet.py')

    import wideresnet
    model = wideresnet.resnet18(num_classes=365)
    checkpoint = torch.load(model_file, map_location=lambda storage, loc: storage)
    state_dict = {str.replace(k,'module.',''): v for k,v in checkpoint['state_dict'].items()}
    model.load_state_dict(state_dict)
    
    # hacky way to deal with the upgraded batchnorm2D and avgpool layers...
    for i, (name, module) in enumerate(model._modules.items()):
        module = recursion_change_bn(model)
    model.avgpool = torch.nn.AvgPool2d(kernel_size=14, stride=1, padding=0)
    
    model.eval()



    # the following is deprecated, everything is migrated to python36

    ## if you encounter the UnicodeDecodeError when use python3 to load the model, add the following line will fix it. Thanks to @soravux
    #from functools import partial
    #import pickle
    #pickle.load = partial(pickle.load, encoding="latin1")
    #pickle.Unpickler = partial(pickle.Unpickler, encoding="latin1")
    #model = torch.load(model_file, map_location=lambda storage, loc: storage, pickle_module=pickle)

    model.eval()
    # hook the feature extractor
    features_names = ['layer4','avgpool'] # this is the last conv layer of the resnet
    for name in features_names:
        model._modules.get(name).register_forward_hook(hook_feature)
    return model

In [113]:
# load the labels
classes, labels_IO, labels_attribute, W_attribute = load_labels()

In [114]:
# load the model
features_blobs = []
model = load_model()

In [115]:
# load the transformer
tf = returnTF() # image transformer

In [116]:
# get the softmax weight
params = list(model.parameters())
weight_softmax = params[-2].data.numpy()
weight_softmax[weight_softmax<0] = 0

In [127]:
directory = os.fsencode("/content/gdrive/MyDrive/digital_humanities/Walkthroughts/Siedler_Frames/")

frame_names = []
io_score = []
io = []
prob_cat_1 = []
cat_1 = []
prob_cat_2 = []
cat_2 = []
prob_cat_3 = []
cat_3 = []
prob_cat_4 = []
cat_4 = []
prob_cat_5 = []
cat_5 = []

for file in os.listdir(directory):

    filename = os.fsdecode(file)

    frame_names.append(filename)

    img = Image.open("/content/gdrive/MyDrive/digital_humanities/Walkthroughts/Siedler_Frames/" + filename)
    input_img = V(tf(img).unsqueeze(0))   
    logit = model.forward(input_img)
    h_x = F.softmax(logit, 1).data.squeeze()
    probs, idx = h_x.sort(0, True)
    probs = probs.numpy()
    idx = idx.numpy() 

    io_image = np.mean(labels_IO[idx[:10]])
    io_score.append(io_image)
    if io_image < 0.5:
      io.append("indoor")
    else:
      io.append("outdoor")
    
    prob_cat_1.append(probs[0])
    cat_1.append(classes[idx[0]])
    prob_cat_2.append(probs[1])
    cat_2.append(classes[idx[1]])
    prob_cat_3.append(probs[2])
    cat_3.append(classes[idx[2]])
    prob_cat_4.append(probs[3])
    cat_4.append(classes[idx[3]])
    prob_cat_5.append(probs[4])
    cat_5.append(classes[idx[4]])

    img = cv2.imread("/content/gdrive/MyDrive/digital_humanities/Walkthroughts/Siedler_Frames/" + filename)
    loc1 = "Location 1: {} ({})". format(cat_1[len(cat_1)-1], "{:.2f}".format(prob_cat_1[len(prob_cat_1)-1]))
    loc2 = "Location 2: {} ({})". format(cat_2[len(cat_2)-1], "{:.2f}".format(prob_cat_2[len(prob_cat_2)-1]))
    loc3 = "Location 3: {} ({})". format(cat_3[len(cat_3)-1], "{:.2f}".format(prob_cat_3[len(prob_cat_3)-1]))
    cat = "Category: {}".format(io[len(io)-1])

    color = (255,255,255)
    cv2.putText(img, loc1, (20, 40), cv2.FONT_HERSHEY_SIMPLEX,0.5,color,1,cv2.LINE_AA, )
    cv2.putText(img, loc2, (20, 60), cv2.FONT_HERSHEY_SIMPLEX,0.5,color,1,cv2.LINE_AA, )
    cv2.putText(img, loc3, (20, 80), cv2.FONT_HERSHEY_SIMPLEX,0.5,color,1,cv2.LINE_AA, )
    cv2.putText(img, cat, (20, 110), cv2.FONT_HERSHEY_SIMPLEX,0.5,color,1,cv2.LINE_AA, )

    cv2.imwrite("/content/gdrive/MyDrive/digital_humanities/Ergebnisse/Location/Siedler/" + filename, img)

In [118]:
df = pd.DataFrame(list(zip(frame_names, io_score, io, prob_cat_1, cat_1, prob_cat_2, cat_2, prob_cat_3, cat_3, prob_cat_4, cat_4, prob_cat_5, cat_5)))
df.columns = ['Name', 'io_score', 'io', 'prob_cat_1', 'cat_1', 'prob_cat_2', 'cat_2', 'prob_cat_3', 'cat_3', 'prob_cat_4', 'cat_4', 'prob_cat_5', 'cat_5']
df.to_csv("/content/gdrive/MyDrive/digital_humanities/Ergebnisse/Location/siedler_location.csv")